# CogniCity: Teste com Lote Retangular (25x50m) e Parâmetros Reais

Este notebook demonstra a execução da pipeline `CogniCity` para um lote retangular "fake" com dimensões de 25m (frente) por 50m (profundidade). Utilizaremos parâmetros urbanísticos realistas para simular um cenário comum.

**Passos:**
1. Configuração do ambiente e imports.
2. Definição da geometria do lote e suas faces.
3. Atribuição de parâmetros urbanísticos realistas.
4. Criação de arquivos GeoJSON temporários para o lote e faces.
5. Criação de um arquivo de configuração YAML específico para este teste.
6. Execução da pipeline `CogniCity`.
7. Carregamento e visualização dos resultados gerados.
8. Limpeza dos arquivos temporários.

## 1. Configuração do Ambiente e Imports

In [ ]:
# Descomente e execute se as bibliotecas não estiverem instaladas
# !pip install geopandas shapely matplotlib pyyaml pandas

In [ ]:
import os
import sys
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString
import matplotlib.pyplot as plt
import yaml
import logging

# Adicionar o diretório raiz do projeto ao PYTHONPATH
# Ajuste o caminho se o seu notebook estiver em uma subpasta diferente de 'notebooks' ou da raiz
project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) # Assume que o notebook está em 'notebooks/'
if 'cogniticy' not in project_root: # Se estiver rodando da raiz do projeto
    project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Raiz do projeto adicionada ao PYTHONPATH: {project_root}")

# Importar módulos do CogniCity
try:
    from cogniticy.pipelines.modeling_pipeline import run_modeling_pipeline
    from cogniticy.core.params import AppParams # Para verificar se o path está ok
    print("Módulos CogniCity importados com sucesso.")
except ImportError as e:
    print(f"Erro ao importar módulos CogniCity: {e}")
    print("Verifique se o PYTHONPATH está configurado corretamente e se a estrutura de pastas do projeto está correta.")
    print(f"Caminhos no sys.path: {sys.path}")

# Configurar logging para ver o output da pipeline
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(module)s - %(message)s')

## 2. Definição da Geometria do Lote Fake (25x50m) e Faces

In [ ]:
# Dimensões do lote
frente_lote = 25.0  # metros
profundidade_lote = 50.0 # metros
id_lote_fake = "LOTEFAKE001"

# Coordenadas do lote (assumindo origem em 0,0 para simplicidade)
# (0,50) --- (25,50)  (FRENTE)
#   |           |
#   |           |
# (0,0)  --- (25,0)   (FUNDOS)
lot_polygon_geom = Polygon([
    (0, 0), (frente_lote, 0), (frente_lote, profundidade_lote), (0, profundidade_lote), (0, 0)
])

# Definir faces
# A convenção de "frente" pode variar. Aqui, vamos assumir que a frente está ao longo do eixo Y maior.
face_frente_geom = LineString([(0, profundidade_lote), (frente_lote, profundidade_lote)])
face_fundos_geom = LineString([(frente_lote, 0), (0, 0)]) # Orientação oposta à frente para consistência
face_lateral_esq_geom = LineString([(0, 0), (0, profundidade_lote)])
face_lateral_dir_geom = LineString([(frente_lote, profundidade_lote), (frente_lote, 0)]) # Orientação oposta à esq.

print(f"Geometria do Lote: {lot_polygon_geom.wkt}")
print(f"Área do Lote: {lot_polygon_geom.area} m² (Esperado: {frente_lote * profundidade_lote})")
print(f"Face Frontal: {face_frente_geom.wkt}")

## 3. Atribuição de Parâmetros Urbanísticos "Reais"

In [ ]:
# Parâmetros urbanísticos realistas (exemplo para uma zona mista)
params_lote_fake = {
    "numlote": id_lote_fake,
    "zot": "ZONA_MISTA_3",
    "codigo": "ZM3",
    "id_quarteirao": "QFAKE01",
    # Normativos
    "max_height": 45.0,  # Altura máxima de 45m
    "max_far": 2.5,      # Coeficiente de Aproveitamento
    "max_lot_coverage": 0.70, # Taxa de Ocupação de 70%
    "gf_floor_height": 4.0, # Pé-direito térreo
    "uf_floor_height": 3.0, # Pé-direito pavimentos tipo
    "min_front_setback": 5.0, # Recuo frontal
    "min_back_setback": 3.0,  # Recuo de fundos
    "min_side_setback": 1.5,  # Recuo lateral (pode ser 0 se geminado, mas vamos usar 1.5)
    "min_setback_start_floor": 4, # Escalonamento a partir do 4º pavimento
    "back_setback_percent": 0.20, # 20% de recuo de fundos por altura acumulada
    # Arquitetônicos
    "min_floor_area": 40.0,
    "min_unit_area": 35.0,
    "target_unit_area": 70.0,
    "core_area_fraction": 0.15,
    "target_efficiency": 0.80,
    # Estacionamento
    "parking_required": True
    # Outros parâmetros podem ser adicionados conforme o PARAMETER_COLUMN_MAPPING
}

print(f"Parâmetros para o lote {id_lote_fake}: {params_lote_fake}")

## 4. Criação de Arquivos GeoJSON Temporários

In [ ]:
temp_data_dir = "./temp_notebook_data"
os.makedirs(temp_data_dir, exist_ok=True)

# CRS para os dados (UTM Zona 22S - Sul do Brasil, exemplo)
crs_metric = "EPSG:31982"

# Criar GeoDataFrame para o lote
lote_gdf_data = [params_lote_fake]
lote_gdf = gpd.GeoDataFrame(lote_gdf_data, geometry=[lot_polygon_geom], crs=crs_metric)
temp_lot_file = os.path.join(temp_data_dir, "lote_fake.geojson")
lote_gdf.to_file(temp_lot_file, driver="GeoJSON")
print(f"Arquivo de lote fake salvo em: {temp_lot_file}")

# Criar GeoDataFrame para as faces
faces_data = [
    {"numlote": id_lote_fake, "tipo": "frente", "geometry": face_frente_geom},
    {"numlote": id_lote_fake, "tipo": "fundos", "geometry": face_fundos_geom},
    {"numlote": id_lote_fake, "tipo": "lateral", "geometry": face_lateral_esq_geom},
    {"numlote": id_lote_fake, "tipo": "lateral", "geometry": face_lateral_dir_geom},
]
faces_geometries = [d.pop('geometry') for d in faces_data]
faces_gdf = gpd.GeoDataFrame(faces_data, geometry=faces_geometries, crs=crs_metric)
temp_faces_file = os.path.join(temp_data_dir, "faces_fake.geojson")
faces_gdf.to_file(temp_faces_file, driver="GeoJSON")
print(f"Arquivo de faces fake salvo em: {temp_faces_file}")

## 5. Criação de Arquivo de Configuração YAML para o Notebook

In [ ]:
temp_output_dir = "./temp_notebook_output"
os.makedirs(temp_output_dir, exist_ok=True)

notebook_config_content = {
    "zoning_parameters": {}, # Deixar vazio para pegar do lote, ou definir defaults globais
    "normative_parameters": { # Defaults globais, serão sobrescritos pelos do lote se existirem
        "max_height": 70.0,
        "max_far": 3.0,
        "max_lot_coverage": 0.75,
        "gf_floor_height": 4.5,
        "uf_floor_height": 3.2,
        "min_front_setback": 3.0,
        "min_back_setback": 2.0,
        "min_side_setback": 0.0, # Exemplo de geminação
        "min_setback_start_floor": 3,
        "back_setback_percent": 0.15
    },
    "architectural_parameters": {
        "min_floor_area": 30.0,
        "target_efficiency": 0.75
    },
    "parking_parameters": { "parking_required": False },
    "modeling_strategy": {
        "h3_resolution": 10,
        "modeling_mode": "advanced", # Para testar o GridSearchOptimizer
        "optimization_objective": "maximize_far_within_height",
        "grid_search_parameters": {
            "shape_ratio_steps": [0.4, 0.6] # Menos passos para um teste rápido
        }
    },
    "simulation_parameters": {
        "project_name": "notebook_fake_lot_test",
        "input_lots_geojson": temp_lot_file,
        "input_block_faces_geojson": temp_faces_file,
        "faces_lot_id_column": "numlote",
        "faces_type_column": "tipo",
        "output_directory": temp_output_dir,
        "output_crs": "EPSG:4326", # Saída em WGS84
        "default_processing_crs": crs_metric, # CRS usado para criar os dados fake
        "lot_id_column_name": "numlote",
        "log_level": "INFO"
    }
}

temp_config_dir = "./temp_notebook_config"
os.makedirs(temp_config_dir, exist_ok=True)
notebook_config_file = os.path.join(temp_config_dir, "notebook_test_config.yaml")

with open(notebook_config_file, 'w') as f:
    yaml.dump(notebook_config_content, f, default_flow_style=False, sort_keys=False)

print(f"Arquivo de configuração do notebook salvo em: {notebook_config_file}")

## 6. Execução da Pipeline CogniCity

In [ ]:
if 'run_modeling_pipeline' in globals():
    print(f"\nExecutando a pipeline com config: {notebook_config_file}...")
    run_modeling_pipeline(config_path=notebook_config_file)
    print("\nExecução da pipeline concluída.")
else:
    print("Função 'run_modeling_pipeline' não foi importada. Verifique os imports e o PYTHONPATH.")

## 7. Carregamento e Visualização dos Resultados

In [ ]:
def plot_geojson_floors(file_path, title):
    if not os.path.exists(file_path):
        print(f"Arquivo não encontrado: {file_path}")
        return None
    try:
        gdf = gpd.read_file(file_path)
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        gdf.plot(ax=ax, alpha=0.7, cmap='viridis', legend=True, edgecolor='black')
        ax.set_title(title)
        ax.set_xlabel("Leste (m)")
        ax.set_ylabel("Norte (m)")
        ax.axis('equal')
        plt.show()
        return gdf
    except Exception as e:
        print(f"Erro ao plotar {file_path}: {e}")
        return None

# Visualizar pavimentos da baseline
baseline_floors_file = os.path.join(temp_output_dir, f"{notebook_config_content['simulation_parameters']['project_name']}_{id_lote_fake}_baseline_floors.geojson")
print(f"\nVisualizando: {baseline_floors_file}")
baseline_floors_gdf = plot_geojson_floors(baseline_floors_file, f"Pavimentos Baseline - Lote {id_lote_fake}")
if baseline_floors_gdf is not None:
    print("Dados dos pavimentos baseline:")
    print(baseline_floors_gdf[['floor_number', 'floor_name', 'base_height', 'floor_area', 'has_setback']].head())

# Visualizar pavimentos da melhor forma
best_shape_floors_file = os.path.join(temp_output_dir, f"{notebook_config_content['simulation_parameters']['project_name']}_{id_lote_fake}_best_shape_floors.geojson")
print(f"\nVisualizando: {best_shape_floors_file}")
best_shape_floors_gdf = plot_geojson_floors(best_shape_floors_file, f"Pavimentos Best Shape - Lote {id_lote_fake}")
if best_shape_floors_gdf is not None:
    print("Dados dos pavimentos da melhor forma:")
    print(best_shape_floors_gdf[['floor_number', 'floor_name', 'base_height', 'floor_area', 'has_setback', 'shape_name']].head())


## 8. Exibir Resumo CSV

In [ ]:
summary_csv_file = os.path.join(temp_output_dir, f"{notebook_config_content['simulation_parameters']['project_name']}_summary_results.csv")
if os.path.exists(summary_csv_file):
    summary_df = pd.read_csv(summary_csv_file)
    print(f"\nResumo da Simulação (do arquivo {summary_csv_file}):")
    # Exibir todas as colunas
    pd.set_option('display.max_columns', None)
    print(summary_df)
else:
    print(f"Arquivo de resumo CSV não encontrado: {summary_csv_file}")

## 9. Limpeza (Opcional)

Descomente as linhas abaixo para remover os arquivos e diretórios temporários criados por este notebook.

In [ ]:
# import shutil

# print("\nLimpando arquivos temporários...")
# try:
#     if os.path.exists(temp_data_dir):
#         shutil.rmtree(temp_data_dir)
#         print(f"Diretório removido: {temp_data_dir}")
#     if os.path.exists(temp_config_dir):
#         shutil.rmtree(temp_config_dir)
#         print(f"Diretório removido: {temp_config_dir}")
#     if os.path.exists(temp_output_dir):
#          shutil.rmtree(temp_output_dir)
#          print(f"Diretório removido: {temp_output_dir}")
# except Exception as e:
#     print(f"Erro ao limpar arquivos temporários: {e}")